In [1]:
from __future__ import print_function
#*********** Psi4 Drivers
import psi4.core as core
#********** OpenMM Drivers
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
#*********** QM/MM classes
from QM_MM_classes import *
#******** this is module that goes with sapt force field files to generate exclusions
from sapt_exclusions import *
#***************************
from routines import trim_shift_PME_grid
import numpy as np
import scipy as scipy
# other stuff
from sys import stdout
from time import gmtime, strftime
from datetime import datetime

In [2]:
QMatoms=(0,)
# QM atoms and MM atoms with analytic Coulomb embedding
# note that QMregion should be ordered ( QMatoms , ... ), otherwise method set_geometry won't work properly.
# we currently don't have a check for this, as we anticipate that we will eventually generate QMregion automatically
# from QMatoms using a cutoff distance
QMregion=(0,1,2,3)

# Make sure QMatoms is subset of QMregion
if not set(QMatoms).issubset(QMregion) :
   print(' QMatoms must be subset of QMregion !!')
   sys.exit()

In [3]:
DFT_functional='PBE'

# *********************************************************************
#                     Create MM system object
#**********************************************************************

# Initialize: Input list of pdb and xml files, and QMregion
MMsys=MM( pdb_list = [ './input_files/He_5ions.pdb', ] , residue_xml_list = [ './input_files/sapt_residues.xml' , ] , ff_xml_list = [ './input_files/sapt.xml', ] , QMregion = QMregion  )

# if periodic residue, call this
#MMsys.set_periodic_residue(True)

# set PME parameters.  This is important for control of accuracy for vext interpolation to DFT quadrature
# choice of alpha:  For n=43 grid, 60 Angstrom box, OpenMM chooses alpha= 2.389328 nm^-1
#MMsys.setPMEParameters( pme_alpha=3.0 , pme_grid_a=60 , pme_grid_b=60 , pme_grid_c=60 ) # alpha, nx, ny, nz
MMsys.setPMEParameters( pme_alpha=2.0 , pme_grid_a=80 , pme_grid_b=80 , pme_grid_c=80 ) # alpha, nx, ny, nz

#***********  Initialze OpenMM API's, this method creates simulation object
MMsys.set_platform('Reference')   # only 'Reference' platform is currently implemented!

# IMPORTANT: generate exclusions for SAPT-FF
sapt_exclusions = sapt_generate_exclusions(MMsys.simmd, MMsys.system, MMsys.modeller.positions)

# Umbrella potential on QM atoms
#MMsys.setumbrella( 'N2', 'grph', 'C100', 2000.0 , 0.4 )   # molecule1, molecule2, atom2,  k (kJ/mol/nm^2) , r0 nm

In [4]:
# *********************************************************************
#                     Create QM system object
#**********************************************************************

# Define QM region and Initialize QM class
# possible quadrature grids: see Psi4 manual
#quadrature_grid = ( 50 , 12 )  # spherical points, radial points
quadrature_grid = ( 302 , 75 )  # spherical points, radial points

QMsys = QM( QMname = 'test' , basis = 'aug-cc-pvdz' , dft_spherical_points = quadrature_grid[0] , dft_radial_points = quadrature_grid[1] , scf_type = 'df' , qmmm='true' )

# Fill QM region with atoms.  Use MMsys to get element types
QMsys.set_QM_region( MMsys, QMregion, QMatoms )

# initial QM energy, Get QM positions from MMsystem
positions  = MMsys.get_positions_QM( QMsys )

# set geometry of QM region
QMsys.set_geometry( positions , charge = 0 )

In [8]:
#**********************************************************************
#                     QM/MM Simulation
#**********************************************************************
# Get QM positions from MMsystem
positions  = MMsys.get_positions_QM( QMsys )

#******************* External potential on PME grid ******************
state = MMsys.simmd.context.getState(getEnergy=True,getForces=True,getVelocities=True,getPositions=True,getVext_grids=True, getPME_grid_positions=True)

# external potential on PME grid
vext_tot = state.getVext_grid()
# PME grid positions
PME_grid_positions = state.getPME_grid_positions()
#** vext_tot from OpenMM is in kJ/mol/e  units, convert to Hartree for input to Psi4
vext_tot = np.array( vext_tot ) / 2625.4996
#** PME_grid_positions from OpenMM is in nanometers, convert to Bohr for input to Psi4
PME_grid_positions = np.array( PME_grid_positions ) * 18.89726

# update positions on QMsys
QMsys.set_geometry( positions , charge = 0 )


In [9]:
PME_grid_positions

array([[  0.       ,   0.       ,   0.       ],
       [  0.       ,   0.       ,   1.4172945],
       [  0.       ,   0.       ,   2.834589 ],
       ...,
       [111.9662655, 111.9662655, 109.1316765],
       [111.9662655, 111.9662655, 110.548971 ],
       [111.9662655, 111.9662655, 111.9662655]])

In [14]:
x1=PME_grid_positions[:,2]

In [15]:
x1=x1[0:80]

In [28]:
test=np.array([[ 4 , 4 ,4] , [ 28 , 30 , 69], [ 13 , 4 , 54]])

In [29]:
grid=(x1 , x1 , x1)

In [30]:
vext_tot


array([0.00178386, 0.00183693, 0.00189106, ..., 0.00169877, 0.00165202,
       0.00160651])

In [31]:
vext3d = numpy.reshape(vext_tot, (80,80,80))

In [32]:
values_x = scipy.interpolate.interpn( grid , vext3d , test, method='linear', bounds_error=True)

In [33]:
values_x

array([ 0.00157447, -0.00561606, -0.00010296])

In [34]:
values_x1 = scipy.interpolate.griddata( PME_grid_positions , vext_tot , test , method='linear' )

In [35]:
values_x1

array([ 0.00157601, -0.00561649, -0.00010232])